This document is work of Anand Chauhan, E21CSEU0173.

Welcome to my work. This is an interactive python notebook meant to document my approach and work for the Law Internship Project. I will be writing down all the approach methods and thoughts I had in order to figure out how to work with the provided data.

In [ ]:
# we begin by loading the file into this notebook.
# ***DO NOT RERUN THIS NOTEBOOK IF YOU DO NOT HAVE THE FILE UPLOADED 
# TO THE SESSION OR IN THE SAME DIRECTORY IN CASE OF DOWNLOADED IPYNB FILES***
import numpy as np
import pandas as pd
import re

megadoc = pd.read_json('processed.json').astype(str)
megadoc.head()

,CaseNo,Judgement,ArgumentsOfPetitioner,ArgumentsOfResponder,Facts,Principle,Statute,CaseCited,Reasoning,Decision,NoOfJudges,Decision_Status
0,Civil Appeal No. 6006 Of 2001 (With C.A. No. 3...,Dharmadhikari J.\t\tThese two cross appeals ha...,32. It needs to be mentioned at this stage tha...,44. The alternative argument advanced on behal...,1.These two cross appeals have been preferred ...,1.These two cross appeals have been preferred ...,"Specific Relief Act, 1963 Section 20&nbsp;",Lala Durga Prasad vs. Lala Deep Chand. (1954 S...,After hearing the argument at length advanced ...,&nbsp;70. On duly appreciating of the evidence...,2,2
1,Civil Appeal No. 4674 Of 1998 (With C.A. No. 4...,"Srikrishna, J.1.&nbsp;Leave granted in the spe...",&nbsp;Learned counsel for the appellants conte...,&nbsp;Learned counsel for the State Government...,The appellants manufacture gutka within the st...,&nbsp;The common issue raised for consideratio...,"Prevention of Food Adulteration Act, 1954 - Se...",Gandhi Irwin Salt Manufacturers Association vs...,There appears to be merit in the contentions o...,"In the result, we allow the appeals and the wr...",2,1
2,Civil Appeal No. 8452 Of 2003 (With C.A. No. 8...,"Arijit Pasayat, J.:-1. Leave granted.2. Appell...","&nbsp;On behalf of the appellant-workmen, rely...","7. Per contra, on behalf of the respondent Man...",2. Appellants contend that the view which was ...,2. Appellants contend that the view which was ...,Maharashtra Recognition of Trade Unions &amp; ...,1.Steel Authority of India Ltd. and Ors. Vs. N...,CIPLA&#39;s case (supra) where detailed analys...,These appeals are without merit and deserve di...,2,0
3,Civil Appeal No. 6790 Of 2003,"H.K. Sema, J.1. This appeal is directed agains...","24. Mr. Rohtagi, learned senior counsel, has d...","12. Dr. Singhvi, learned senior counsel appear...",3. The 1st appellant is the wife of the 5th re...,This appeal is directed against the judgment o...,"Code of Civil Procedure, 1908 -","Vidhyadhar vs. Manikrao and Another, (1999) 3 ...",15. Having regard to the directions in the ord...,"Accordingly, the appeal fails and is dismissed...",2,2
4,Civil Appeal No. 439 Of 1997 (With C.A. No. 84...,"S.N. Variava, J.:-1. Leave granted.2. Both the...",13. Written submissions on behalf of the Appel...,In this behalf of the relevant portions of the...,"On 28th of August, 1993, the Appellants appear...",\tWhether a co-operative society registered un...,"Banking Regulation Act, 1949 - Section 22&nbsp...",\tR.C. Cooper vs. Union of India reported in (...,17. We are unable to accept these submissions ...,In view of what we have held we direct the RBI...,2,1


reading through some of the data provides the following insights:
- the data has a lot of encoding artifacts (such as the occasional `&nbsp;` or the `&amp;`)
- the entire case seems like a list of paragraphs, whether it be arguments, facts, or even the judgements

while we could work with the data by considering each of those paragraphs indiviually, we would need to first clean the data. I have worked on two small AI projects before though so this is completely alien to me. I will not be spending a lot of time on cleaning the data itself however, because I think that getting an idea of the approach is more important.

In [ ]:
# creating a small cleaning function that I can use to simply apply to every cell in the megadoc
def clean(x):
    text = str(x)
    # replacing every artifact with the character(s) it represents, 
    # and replacing every inconvenient character
    # with something subjectively cleaner
    text = re.sub("&nbsp;", " ", text)
    text = re.sub(". Vs. ", " v ", text)
    text = re.sub(r"\t", "   ", text)
    text = re.sub(":-", "", text)
    text = re.sub(";", " ", text)
    text = re.sub("&#39;", "'", text)
    text = re.sub("&#39 ", "'", text)
    result = re.sub("&amp;", "&", text)

    return result
    

# tests a rawstring to filter.
# note: the tab sequence \s doesn't work here but works on the dataframe somehow.
test = r"&nbsp;70. On duly appreciating :-of the evid\tence, the Steel Authority of India Ltd. and Ors. Vs. N..."
print(clean(test))

 70. On duly appreciating of the evid\tence, the Steel Authority of India Ltd. and Ors v N...


In [ ]:
megadoc = megadoc.applymap(clean)
megadoc.head()

,CaseNo,Judgement,ArgumentsOfPetitioner,ArgumentsOfResponder,Facts,Principle,Statute,CaseCited,Reasoning,Decision,NoOfJudges,Decision_Status
0,Civil Appeal No. 6006 Of 2001 (With C.A. No. 3...,Dharmadhikari J. These two cross appeals ...,32. It needs to be mentioned at this stage tha...,44. The alternative argument advanced on behal...,1.These two cross appeals have been preferred ...,1.These two cross appeals have been preferred ...,"Specific Relief Act, 1963 Section 20",Lala Durga Prasad vs. Lala Deep Chand. (1954 S...,After hearing the argument at length advanced ...,70. On duly appreciating of the evidence on r...,2,2
1,Civil Appeal No. 4674 Of 1998 (With C.A. No. 4...,"Srikrishna, J.1. Leave granted in the special ...",Learned counsel for the appellants contend th...,Learned counsel for the State Governments con...,The appellants manufacture gutka within the st...,The common issue raised for consideration of ...,"Prevention of Food Adulteration Act, 1954 - Se...",Gandhi Irwin Salt Manufacturers Association vs...,There appears to be merit in the contentions o...,"In the result, we allow the appeals and the wr...",2,1
2,Civil Appeal No. 8452 Of 2003 (With C.A. No. 8...,"Arijit Pasayat, J.1. Leave granted.2. Appellan...","On behalf of the appellant-workmen, relying u...","7. Per contra, on behalf of the respondent Man...",2. Appellants contend that the view which was ...,2. Appellants contend that the view which was ...,Maharashtra Recognition of Trade Unions &amp ...,1.Steel Authority of India Ltd. and Ors v Nati...,CIPLA's case (supra) where detailed analysis h...,These appeals are without merit and deserve di...,2,0
3,Civil Appeal No. 6790 Of 2003,"H.K. Sema, J.1. This appeal is directed agains...","24. Mr. Rohtagi, learned senior counsel, has d...","12. Dr. Singhvi, learned senior counsel appear...",3. The 1st appellant is the wife of the 5th re...,This appeal is directed against the judgment o...,"Code of Civil Procedure, 1908 -","Vidhyadhar vs. Manikrao and Another, (1999) 3 ...",15. Having regard to the directions in the ord...,"Accordingly, the appeal fails and is dismissed...",2,2
4,Civil Appeal No. 439 Of 1997 (With C.A. No. 84...,"S.N. Variava, J.1. Leave granted.2. Both these...",13. Written submissions on behalf of the Appel...,In this behalf of the relevant portions of the...,"On 28th of August, 1993, the Appellants appear...",Whether a co-operative society registered u...,"Banking Regulation Act, 1949 - Section 22 Nati...",R.C. Cooper vs. Union of India reported in ...,17. We are unable to accept these submissions ...,In view of what we have held we direct the RBI...,2,1


The approach I have managed to think of in the given limited time is as follows:
- The primary goal is to understand how to achieve the outcome of a given legal case based on what information is available before the case actually goes to trial. This means that certain columns in the data would be unavailable initially, and our approach must not use them for more than the scope of training an AI model or something similar.
- Upon some minor research (source: [this blogpost, mostly](https://philosophicaldisquisitions.blogspot.com/2021/03/understanding-legal-argument-1-five.html)), it is revealed that the base of all legal arguments is the arguments that certain facts support or deny certain claims. although i do not know if indian law works in the exact same fashion, the blogpost does not seem to be aligned to a specific legal system. 
  > There are lots of potential legal rules that could apply to a given set of facts. And there are lots of qualifications and exceptions to legal rules. You have to argue for the rules themselves and show why a particular rule (or major premise) should apply to a particular case.
- With this in mind, I believe that the arguments of a case can be referred to as weighted events, with their impact on the case ranging from -1 (completely in favour of the respondant/prosecution) to 1 (completely in favour of the plantiff/defedant/petitioner). These arguments can then be further graded based on their relevance to the judgement with the influence of factors such as evidence, facts, or cited cases. I do not know the specific of the AI that would go behind this, but I believe that by introducing a scoring system like this, we could predict what side the judgement is going to lean towards before a case goes to court.
- The grading system could also be in the _[-1, 1]_ interval, since a really badly supported argument could potentially sway the judgement in the opposition's favour (eg. a perjury, although those can lead to mistrials too). These scores could then be added up and then the expected leaning position could be calculated. Similar to expected values in a probabalistic calculation but with weights instead. This system can go extremely in depth as well, where arguments that are mutually exclusive and completely oppose each other will have their scores inversely related to each other. The overall score of the case then will decide who the arguments and facts currently favour.
- The drawbacks I can think of for this approach would be: 
  - It could be difficult to pinpoint points of failure in a legal case, so this approach provides improper legal advice.
  - It would be difficult to generate proper decision text as in the sample, unless a good deal of depth into the working of the AI is established (what fact supports/denies what arguments is known, and how similar the cited case was).
  - The training data provided is relatively less, and could result in overfitting of some kind.
  - Some arguments may evolve over the course of a trial, their impact on the overall judgement varing greatly as a result. I do not know if there is a need to account for changing information, but it would require running the entire process from the beginning every time. This can get tedious
 
I will now demonstrate something to the effect of my approach, but with completely random numbers. I will take a random case, split out all the arguments, and then grade, score and add them to provide a working example of the approach I have in mind. However, no AI will be involved in this, and the generated numbers will all be random. Also, no involvement of the facts or the citiations will be required here, which should obviously be paramount in the actual application.

In [ ]:
# first, we select a random case and make a numpy array out of its arguments.
case_no = int(np.random.randint(0, 30, 1))
dirtyargs = np.array(megadoc.loc[case_no, ['ArgumentsOfPetitioner', 'ArgumentsOfResponder']])
dirtyargs

array(["13. Written submissions on behalf of the Appellants given by Mr. Andhyarujina also need to be noted. The relevant portion reads as follows: According to the scheme of the BR Act, for the grant of a license to the Appellant, it has to first come within the meaning of a 'cooperative bank' i.e. either a state cooperative bank, central cooperative bank or a primary cooperative bank. It is only after a notification under section 2(u) of NABARD is issued the cooperative society becomes a cooperative bank within the meaning of section 5(cci) of the BR Act as amended by section 56(c) of the BR Act and thus became eligible to for license from the RBI under section 22 of the BR act read with section 56(o) of the BR Act. Thus a notification under section 2(u) in these circumstances necessarily precedes the grant of a license by the RBI. 29. It must be mentioned that it was submitted by Mr. Andhyarujina that a co-operative society registered under the Co-operative Societies Act, 1912 can o

In [ ]:
# making a regex pattern to find argument numbers. this needs some tweaking, but works for the most part.
pattern = r"([1-9][0-9]\.\s)+|([0-9]\.\s)"
# create an empty array for use in a DF later.
cleanargs = np.array([])
cleanargs = np.append(cleanargs, re.split(pattern, dirtyargs[0]))
cleanargs = np.append(cleanargs, re.split(pattern, dirtyargs[1]))

# remove all empty elements, or elements that only have the matched number.
cleanargs = cleanargs[cleanargs != np.array(None)]
cleanargs = np.array([i for i in cleanargs if len(i) > 6])

cleanargs

array(["Written submissions on behalf of the Appellants given by Mr. Andhyarujina also need to be noted. The relevant portion reads as follows: According to the scheme of the BR Act, for the grant of a license to the Appellant, it has to first come within the meaning of a 'cooperative bank' i.e. either a state cooperative bank, central cooperative bank or a primary cooperative bank. It is only after a notification under section 2(u) of NABARD is issued the cooperative society becomes a cooperative bank within the meaning of section 5(cci) of the BR Act as amended by section 56(c) of the BR Act and thus became eligible to for license from the RBI under section 22 of the BR act read with section 56(o) of the BR Act. Thus a notification under section 2(u) in these circumstances necessarily precedes the grant of a license by the RBI. ",
       'It must be mentioned that it was submitted by Mr. Andhyarujina that a co-operative society registered under the Co-operative Societies Act, 1912 ca

In [ ]:
# initializing a sample dataframe, where IMPACT = WEIGHT*SCORE.
# summation of impact should give us an idea of who the arguments favour.
# let's call this variable the sigma. if sigma > 0, then the arguments favour the petitioner
# if sigma < 0, then the arguments favour the respondant.
sample = pd.DataFrame(columns=['Arguments', 'Weight', 'Score', 'Impact'])
sigma = float()

Some caveats 
- **WEIGHT:** The 'weight' column here refers to how important an argument is to the case at hand. This can be determined by the actual Law database, or even the cited case. The similarity to important arguments in the cited case, or things that directly mention claims that are also part of the law in general. More factors could be added here, and weights could be linked. EX - An argument asserting that someone committed a murder could be linked to another argument stating that he did not have an alibi at the time of the murder.
- **SCORE:** The 'score' column here refers to how much water a given argument can hold based on evidence, facts, and even cited case data. Just like the weight column, score of arguments can be linked too. EX - Just like the above example, if there is an argument stating that the defendant did in fact have an alibi, then the value of the argument stating he did not will be the inverse of the value stating that he did.
- **IMPACT = SCORE x WEIGHT** (subject to change, I am not an expert): The impact that an argument could have over the judgement. Summation of these will provide an idea of whose side the inital arguments favor.
- **SIGMA:** my term for the summation of the impacts of all arguments. a mean could also work here, but I have not used it in order to find the absolute value of the leaning of the arguments, if that makes any sense.

In [ ]:
# now we insert the cleanargs array into the argument column, 
# populate the weight and score columns with random numbers between 0 and 1,
# and set impact = score*weight as discussed in the above cell.
size = len(cleanargs)
sample['Arguments'] = cleanargs.tolist()
sample['Weight'] = np.array(np.random.uniform(-1, 1, size=size))
sample['Score'] = np.array(np.random.uniform(-1, 1, size=size))
sample['Impact'] = sample['Weight']*sample['Score']
sample.head()

,Arguments,Weight,Score,Impact
0,Written submissions on behalf of the Appellant...,0.192351,0.224486,0.043180
1,It must be mentioned that it was submitted by ...,0.199437,0.595647,0.118794
2,It was submitted that the activities of accept...,-0.576093,-0.763673,0.439947
3,Faced with this situation it was submitted tha...,-0.051205,-0.047875,0.002451
4,Faced with this situation Mr. Anandhyarujina s...,-0.358281,0.624241,-0.223654


In [ ]:
# basic output, this could be improved a lot 
# based on how the AI works, but that is outside 
# the scope of this notebook.
sigma = np.sum(sample['Impact'])
if (sigma < 0):
    print("For Case " + megadoc.loc[case_no, "CaseNo"] + ": The arguments favour the defense.")
else:
    print("For Case " + megadoc.loc[case_no, "CaseNo"] + ": The arguments favour the plantiff.")

For Case Civil Appeal No. 439 Of 1997 (With C.A. No. 8478 Of 2003): The arguments favour the plantiff.


This is all from my side. The premise of the project is very interesting to me and I would love to contribute in any way I can to it. Thank you for your time.

In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]